In [ ]:
import ipyvolume as ipv
import SimpleITK as sitk
import numpy as np
import os

def load_dicom_series(directory_path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(directory_path)
    reader.SetFileNames(dicom_names)
    image = reader.Execute()
    return image

def display_images(image):
    # Convert the SimpleITK image to a numpy array
    image_array = sitk.GetArrayFromImage(image)
    
    # Normalize to 0-255
    image_array = ((image_array - image_array.min()) * (1/(image_array.max() - image_array.min()) * 255)).astype('uint8')
    
    # Now plot the volume!
    ipv.quickvolshow(image_array, level=[0.25, 0.75], opacity=0.03, level_width=0.1, data_min=0, data_max=255)

# Paths to the DICOM directories
fixed_image_path = r"C:\Users\HP\Documents\GitHub\3D-3D_Image_Registration\SE000000_fixed"
moving_image_path = r"C:\Users\HP\Documents\GitHub\3D-3D_Image_Registration\SE000003_moving"

# Load the DICOM images
fixed_image = load_dicom_series(fixed_image_path)
moving_image = load_dicom_series(moving_image_path)

# Display the images
display_images(fixed_image)
display_images(moving_image)

# Apply the transformation
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMeanSquares()
registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=100, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
registration_method.SetInterpolator(sitk.sitkLinear)

initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_image, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)

registration_method.SetInitialTransform(initial_transform, inPlace=False)
final_transform_v1 = registration_method.Execute(sitk.Cast(fixed_image, sitk.sitkFloat32), 
                                                 sitk.Cast(moving_image, sitk.sitkFloat32))

# Resample the moving image onto the fixed image's grid
resampler = sitk.ResampleImageFilter()
resampler.SetReferenceImage(fixed_image);
resampler.SetTransform(final_transform_v1);
resampled_moving_image = resampler.Execute(moving_image)fol

# Display the images
display_images(fixed_image)
display_images(resampled_moving_image)


In [ ]:
import os
import pydicom
import cv2
import numpy as np

# Your current path to the image
data_path = r"C:\Users\HP\Documents\GitHub\3D-3D_Image_Registration\SE000003"

# Adjust this to point to one specific image file
image_path = os.path.join(data_path, "CT000000")

# Read DICOM file
ds = pydicom.dcmread(image_path)

# Print some attributes
print(f"Patient ID: {ds.PatientID}")
print(f"Modality: {ds.Modality}")
print(f"Image Size: {ds.Rows} x {ds.Columns}")

# Preprocessing
image = ds.pixel_array  # Get pixel data
image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)  # Normalize pixel values between 0 and 255

# Show the image
import matplotlib.pyplot as plt
plt.imshow(image, cmap=plt.cm.bone)
plt.show()


In [ ]:
# import os
# import pydicom
# import matplotlib.pyplot as plt

# def load_scan(path):
#     slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
#     slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
#     return slices

# def plot_images(images):
#     # Plot all the DICOM images
#     for i in range(len(images)):
#         plt.figure(figsize=(5,5))
#         plt.imshow(images[i].pixel_array, cmap=plt.cm.bone)
#         plt.show()

# data_dir = "C:\\Users\\HP\\Documents\\GitHub\\3D-3D_Image_Registration\\SE000003"
# dicom_images = load_scan(data_dir)
# plot_images(dicom_images)


In [ ]:

# Define path to data
data_path = r"C:\Users\HP\Documents\GitHub\3D-3D_Image_Registration\SE000003"

# Load DICOM files
def load_dicom_files(path):
    dicom_files = []
    for i in range(426):  # Adjust this based on the total files you have
        filename = f"CT{str(i).zfill(6)}"
        dicom_files.append(pydicom.dcmread(os.path.join(path, filename)))
    return dicom_files

dicom_files = load_dicom_files(data_path)

# Visualize a DICOM image
plt.imshow(dicom_files[0].pixel_array, cmap=plt.cm.bone)
plt.title("DICOM Image")
plt.show()

# Visualize a stack of DICOM images
# NOTE: You should adjust the following code based on the orientation of your images
image_stack = np.stack([file.pixel_array for file in dicom_files])
plt.imshow(image_stack[200], cmap=plt.cm.bone)  # Show a slice in the middle of the stack
plt.title("Slice from DICOM Image Stack")
plt.show()

# Analyze DICOM metadata
metadata = []
for dicom_file in dicom_files:
    metadata.append({
        "PatientAge": dicom_file.PatientAge,
        "PatientSex": dicom_file.PatientSex,
        "SliceThickness": dicom_file.SliceThickness,
        "PixelSpacing": dicom_file.PixelSpacing,
        # Add more fields as required
    })
metadata_df = pd.DataFrame(metadata)
print(metadata_df.describe(include="all"))  # Basic statistics of metadata

# Analyze image statistics
pixel_arrays = [file.pixel_array.ravel() for file in dicom_files]
pixels = np.concatenate(pixel_arrays)
print(f"Pixel intensity mean: {np.mean(pixels)}")
print(f"Pixel intensity standard deviation: {np.std(pixels)}")
print(f"Pixel intensity min: {np.min(pixels)}")
print(f"Pixel intensity max: {np.max(pixels)}")

# Histogram of pixel intensities
plt.hist(pixels, bins=50, color='gray')
plt.title("Histogram of Pixel Intensities")
plt.show()

# Visualizing Slice Thickness
# NOTE: This assumes that slice thickness is the same for all images
plt.figure(figsize=(6, 6))
sns.boxplot(y="SliceThickness", data=metadata_df)
plt.title("Boxplot of Slice Thickness")
plt.show()

# More analysis steps could be added as per your requirements


## wORKING WITH MODELS 

In [ ]:
# Import necessary libraries
import os
import pydicom
import numpy as np
import torch
import torchvision
from torch import nn, optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# Define path to data
data_path = r"C:\Users\HP\Documents\GitHub\3D-3D_Image_Registration\SE000003"

# Read DICOM files
def read_dicom(path):
    dicom_images = []
    for i in range(426): # Adjust this number based on your total files
        file = f"CT{str(i).zfill(6)}"
        dicom_images.append(pydicom.dcmread(os.path.join(path, file)).pixel_array)
    return np.array(dicom_images)

# Preprocess DICOM images
def preprocess(dicom_images):
    # Convert to float32
    dicom_images = dicom_images.astype(np.float32)
    # TODO: implement other preprocessing steps
    # Normalize, resize, remove noise etc.
    return dicom_images


# Create a Dataset
class CTScanDataset(Dataset):
    def __init__(self, dicom_images, transform=None):
        self.dicom_images = dicom_images
        self.transform = transform

    def __len__(self):
        return len(self.dicom_images)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        image = self.dicom_images[idx]
        if self.transform:
            image = self.transform(image)
        return image

# Create a simple U-Net model as a placeholder
# TODO: Replace this with a real U-Net implementation
class UNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv3d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv3d(16, 1, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        return x

# Load DICOM images
dicom_images = read_dicom(data_path)
print(f"Loaded {len(dicom_images)} DICOM images.")

# Preprocess images
preprocessed_images = preprocess(dicom_images)

# Define transformations
transformations = transforms.Compose([
    transforms.ToTensor(),
])

# Create dataset
dataset = CTScanDataset(preprocessed_images, transform=transformations)
print(f"Created dataset with {len(dataset)} images.")

# Split dataset into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)  # Batch size must be 1 for 3D images
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)  # Batch size must be 1 for 3D images

# Initialize model
model = UNet()
print(f"Model has {sum(p.numel() for p in model.parameters())} parameters.")

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
for epoch in range(2):  # loop over the dataset multiple times
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs]
        inputs = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, inputs)  # Assuming we are doing image reconstruction
        loss.backward()
        optimizer.step()

print('Finished Training')
